In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [29]:
# define the hyperparameters
input_size = 8
hidden_size = 10 
output_size = 5
num_layers = 5
arm_size = 3
beta = 0.1
lambd = 0.001

In [129]:
# initialize the parameters

theta = np.random.randn(output_size, 1) / np.sqrt(output_size)
theta = torch.tensor(theta, dtype = torch.float32)

LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)
bb = torch.zeros(LAMBDA.size()[0], dtype=torch.float).view(-1, 1)

In [121]:
# create the model 
import modeldefine
model = modeldefine.Model(input_size, hidden_size, output_size, num_layers).to(device)

tensor([ 8, 10, 10, 10, 10, 10,  5], dtype=torch.int32)


In [142]:
def TRANS(c, a, arm_size):
    #### transfer an array context + action to new context with dimension 2*(__context__ + __armsize__)
    dim = len(c)
    action = np.zeros(arm_size)
    action[a] = 1
    c_final = np.append(c, action)
    c_final = torch.from_numpy(c_final).to(device)
    c_final = c_final.view((len(c_final), 1))
    c_final = c_final.repeat(2, 1)
    c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)
    return c_final

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def get_action(model, context_info, random_indicator):
    ucb = [] 
    for i in range(arm_size):
        temp = TRANS(context_info.cpu(), i, arm_size).to(device)
        feature = model(temp).cpu()

        first_item = torch.mm(feature, theta)
        ucb_item = beta * UCB(LAMBDA, feature)
        ucb.append(first_item + ucb_item)
    
    if random_indicator == 0:
        return ucb.index(max(ucb))
    else:
        return np.random.randint(0, arm_size)

def update_lambda_bb(lambd, bb, context_info, action,reward):
    temp = TRANS(context_info.cpu(), action, arm_size).to(device)
    feature = model(temp).cpu()
    templambda = lambd + torch.mm(feature.t(), feature)
    tempbb = bb + reward * feature.t()
    return templambda, tempbb

def update_theta(lambd, bb):
    return torch.linalg.solve(lambd, bb)

In [122]:
context = torch.randn(1,5).to(device)